In [47]:
import pandas as pd

In [48]:
df=pd.read_csv("/content/driving_log.csv")

In [49]:
df.head()

,center,left,right,steer,throughtle,0,4.23E-06
0,D:\udemy simulator'\beta_simulator_windows\tra...,D:\udemy simulator'\beta_simulator_windows\tra...,D:\udemy simulator'\beta_simulator_windows\tra...,0.0,0.0,0,0.000003
1,D:\udemy simulator'\beta_simulator_windows\tra...,D:\udemy simulator'\beta_simulator_windows\tra...,D:\udemy simulator'\beta_simulator_windows\tra...,0.0,0.0,0,0.000007
2,D:\udemy simulator'\beta_simulator_windows\tra...,D:\udemy simulator'\beta_simulator_windows\tra...,D:\udemy simulator'\beta_simulator_windows\tra...,0.0,0.0,0,0.000003
3,D:\udemy simulator'\beta_simulator_windows\tra...,D:\udemy simulator'\beta_simulator_windows\tra...,D:\udemy simulator'\beta_simulator_windows\tra...,0.0,0.0,0,0.000004
4,D:\udemy simulator'\beta_simulator_windows\tra...,D:\udemy simulator'\beta_simulator_windows\tra...,D:\udemy simulator'\beta_simulator_windows\tra...,0.0,0.0,0,0.000013


In [50]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [51]:
folder_path = '/content/drive/My Drive/car_data/center'

In [59]:
import numpy as np
from PIL import Image
import os

def read_images_into_arrays(folder_path):
    image_arrays = []

    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image = Image.open(os.path.join(folder_path, filename))
            image_array = np.array(image)
            image_arrays.append(image_array)

    return image_arrays

image_arrays = read_images_into_arrays(folder_path)

In [60]:
#split data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    image_arrays,
    df[['steer','throughtle']],
    test_size=0.2,
    random_state=42
)

rotation_train = y_train["steer"]
throttle_train = y_train["throughtle"]
rotation_test = y_test["steer"]
throttle_test = y_test["throughtle"]

In [61]:
rotation_train.shape, throttle_train.shape, rotation_test.shape, throttle_test.shape

((3004,), (3004,), (752,), (752,))

In [62]:
#convert to numpy array
import numpy as np
X_train = np.asarray(X_train)
X_test = np.asarray(X_test)
rotation_train = np.asarray(rotation_train.tolist())
throttle_train = np.asarray(throttle_train.tolist())
rotation_test = np.asarray(rotation_test.tolist())
throttle_test = np.asarray(throttle_test.tolist())

In [63]:
X_train = X_train.transpose(0,3,1,2)
X_test = X_test.transpose(0,3,1,2)

In [64]:
from torch.utils.data import DataLoader, TensorDataset
train_data_steer = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(rotation_train))
test_data_steer = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(rotation_test))

train_loader_steer = DataLoader(train_data_steer, batch_size=32, shuffle=True)
test_loader_steer = DataLoader(test_data_steer, batch_size=32, shuffle=True)


train_data_throughtle = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(rotation_train))
test_data_throughtle = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(rotation_test))

train_loader_throughtle = DataLoader(train_data_throughtle, batch_size=32, shuffle=True)
test_loader_throughtle = DataLoader(test_data_throughtle, batch_size=32, shuffle=True)

In [67]:
import torch
import torch.nn as nn
import torch.optim as optim


In [68]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN_throughtle(nn.Module):
    def __init__(self):
        super(CNN_throughtle, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32 * 40 * 80, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 40 * 80)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x



# Define the loss function and optimizer



In [69]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN_steer(nn.Module):
    def __init__(self):
        super(CNN_steer, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32 * 40 * 80, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 40 * 80)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x



# Define the loss function and optimizer



In [70]:
model_throughtle = CNN_throughtle()
model_steer =CNN_steer()
optimizer_steer = optim.Adam(model_steer.parameters(), lr=0.001)
optimizer_throughtle = optim.Adam(model_throughtle.parameters(), lr=0.001)

In [71]:
loss_function = nn.MSELoss()


In [89]:
train_loss = []
# Training loop
for epoch in range(5):
    train_loss = 0.0

    for batch_x, batch_y in train_loader_steer:
        batch_x = batch_x.float()  # Ensure input data is a float tensor
        batch_y =  batch_y.float() # Ensure target data is a float tensor

        # Zero the parameter gradients
        optimizer_steer.zero_grad()
        # Forward pass through the model
        outputs = model_steer(batch_x)

        # Calculate the loss
        loss_steer = loss_function(outputs, batch_y)

        # Backpropagation and optimization
        loss_steer.backward()
        optimizer_steer.step()
        #Print the loss
        print("Epoch {}: Loss_steer {}".format(epoch, loss_steer.item()))

        train_loss+=loss_steer.item()


    for batch_x, batch_y in train_loader_throughtle:
        batch_x = batch_x.float()  # Ensure input data is a float tensor
        batch_y =  batch_y.float() # Ensure target data is a float tensor

        # Zero the parameter gradients
        optimizer_throughtle.zero_grad()
        # Forward pass through the model
        outputs = model_throughtle(batch_x)

        # Calculate the loss
        loss_throughtle = loss_function(outputs, batch_y)

        # Backpropagation and optimization
        loss_throughtle.backward()
        optimizer_throughtle.step()

        #Print the loss
        print("Epoch {}: Loss_throughtle {}".format(epoch, loss_throughtle.item()))
        train_loss+= loss_throughtle.item()



Epoch 0: Loss_steer 0.12840300798416138
Epoch 0: Loss_steer 0.06823285669088364
Epoch 0: Loss_steer 0.10963524878025055
Epoch 0: Loss_steer 0.05631417781114578
Epoch 0: Loss_steer 0.09417842328548431
Epoch 0: Loss_steer 0.11113676428794861
Epoch 0: Loss_steer 0.0853542685508728
Epoch 0: Loss_steer 0.09193889051675797
Epoch 0: Loss_steer 0.08835306763648987
Epoch 0: Loss_steer 0.057927437126636505
Epoch 0: Loss_steer 0.11115766316652298
Epoch 0: Loss_steer 0.11088228225708008
Epoch 0: Loss_steer 0.13670487701892853
Epoch 0: Loss_steer 0.10729905962944031
Epoch 0: Loss_steer 0.12027847766876221
Epoch 0: Loss_steer 0.1059284582734108
Epoch 0: Loss_steer 0.0709681436419487
Epoch 0: Loss_steer 0.08123031258583069
Epoch 0: Loss_steer 0.07332172989845276
Epoch 0: Loss_steer 0.08987666666507721
Epoch 0: Loss_steer 0.13361170887947083
Epoch 0: Loss_steer 0.08292447030544281
Epoch 0: Loss_steer 0.06357964128255844
Epoch 0: Loss_steer 0.08494628220796585
Epoch 0: Loss_steer 0.05282936990261078
Ep

In [93]:
torch.save(model_steer,"steering_model.pth")
torch.save(model_throughtle,"throughtle_model.pth")